In [2]:
import notebook_common
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    r_word_2400_xlsx,
    w_word_2400_csv,
    w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    pattern_table_written,
    pattern_td_written,
    r_written_freq_nlt_with_enter_txt,
    w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    # w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_tmp_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [3]:
with open(r_written_freq_nlt_with_enter_txt, "r", encoding="utf-8") as f:
    lst = []
    content = f.read()  # 一次性读取
    table_matches = pattern_table_written.findall(content)  # 或 finditer 迭代匹配
    for table_match in table_matches:
        td_matches_lst = pattern_td_written.findall(table_match)
        lst.append(td_matches_lst)
    columns = ["headword", "reading", "roman", "freq", "freq_per", "rank"]
    df = pd.DataFrame(lst, columns=columns)
    to_csv_sig(df, w_written_freq_nlt_with_enter_csv)
    p("w_written_freq_nlt_with_enter_csv--ok")
df

w_written_freq_nlt_with_enter_csv--ok


,headword,reading,roman,freq,freq_per,rank
0,110番,ヒャクトオバン,hyakutooban,108,0.0252,0.69889
1,110番する,ヒャクトオバンスル,hyakutoobansuru,7,0.0016,0.28913
2,1つ,ヒトツ,hitotsu,"4,366",1.0167,0.98121
3,1つする,ヒトツスル,hitotsusuru,2,0.0005,0.14991
4,2ラン,ツーラン,tsuran,99,0.0231,0.68579
...,...,...,...,...,...,...
85106,齲歯,ウバ,uba,19,0.0044,0.41736
85107,龍眼,リュウガン,ryuugan,13,0.0030,0.36012
85108,（有）,ユウゲンガイシャ,yuugengaisha,115,0.0268,0.70698
85109,（社）,シャダンホウジン,shadanhoujin,2,0.0005,0.09486


In [4]:
df.describe()

,headword,reading,roman,freq,freq_per,rank
count,85111,85111,85111,85111,85111,85107
unique,84286,66611,66096,4504,4504,85107
top,する,キコウ,kikou,1,0.0002,0.56826
freq,72,23,23,8074,8074,1


In [5]:
# 指定列中，只要有一列的数据为空，删除该行,Inplace=True
na_columns = ["headword", "reading", "freq", "rank"]
df.dropna(subset=na_columns, inplace=True)
to_deleted_columns = ["rank", "freq_per", "roman"]
# 将无用的列删除
df.drop(to_deleted_columns, axis=1, inplace=True)
# 将词频列的字符串转化为数字
df["freq"] = pd.to_numeric(df["freq"].str.replace(",", ""), errors="coerce")

p(df.describe())
l()
p(df.dtypes)
l()
p(df)

                freq
count   85107.000000
mean      504.548862
std      6804.399540
min         1.000000
25%         6.000000
50%        32.000000
75%       153.000000
max    968064.000000
--------------------------------------------------------------------------------
headword    object
reading     object
freq         int64
dtype: object
--------------------------------------------------------------------------------
      headword    reading  freq
0         110番    ヒャクトオバン   108
1       110番する  ヒャクトオバンスル     7
2           1つ        ヒトツ  4366
3         1つする      ヒトツスル     2
4          2ラン       ツーラン    99
...        ...        ...   ...
85106       齲歯         ウバ    19
85107       龍眼      リュウガン    13
85108      （有）   ユウゲンガイシャ   115
85109      （社）   シャダンホウジン     2
85110      （財）   ザイダンホウジン    49

[85107 rows x 3 columns]


In [56]:
df["count"] = df.groupby(["headword", "reading"]).transform("size")
df

,headword,reading,freq,count
0,110番,ヒャクトオバン,108,1
1,110番する,ヒャクトオバンスル,7,1
2,1つ,ヒトツ,4366,1
3,1つする,ヒトツスル,2,1
4,2ラン,ツーラン,99,1
...,...,...,...,...
85106,齲歯,ウバ,19,1
85107,龍眼,リュウガン,13,1
85108,（有）,ユウゲンガイシャ,115,1
85109,（社）,シャダンホウジン,2,1


In [6]:
# 临时文件
to_csv_sig(df,'d:/tmp/written_freq_tmp1.csv')
p('written_freq_tmp1.csv--ok')

written_freq_tmp1.csv--ok


In [44]:
# 按单词对词频进行汇总
summed_df = df["freq"].groupby([df["headword"],df["reading"]]).sum().to_frame()
summed_df.rename(columns={"freq": "written_freq"}, inplace=True)
summed_df.sort_values(by="written_freq", ascending=False, inplace=True)
to_csv_sig(summed_df, w_written_freq_nlt_sumed_csv, need_index=True)
p(summed_df)
p("w_written_freq_nlt_sumed_csv--ok")

                   written_freq
headword reading               
いる-非自立   イル              968064
こと       コト              701470
の        ノ               686025
する       スル              607405
ある       アル              496737
...                         ...
所思       ショシ                  1
襲用       シュウヨウ                1
襲爵       シュウシャク               1
フラッシングする フラッシングスル             1
フラダンスする  フラダンススル              1

[84644 rows x 1 columns]
w_written_freq_nlt_sumed_csv--ok


In [45]:
summed_df.index

MultiIndex([(  'いる-非自立',       'イル'),
            (      'こと',       'コト'),
            (       'の',        'ノ'),
            (      'する',       'スル'),
            (      'ある',       'アル'),
            (      'なる',       'ナル'),
            (      'その',       'ソノ'),
            (      'よう',       'ヨウ'),
            (      '言う',       'イウ'),
            (      'この',       'コノ'),
            ...
            (     '鼻する',     'ハナスル'),
            (    '鼻ぐすり',    'ハナグスリ'),
            (    '扇情する',  'センジョウスル'),
            (    '所存する',   'ショゾンスル'),
            (      '所司',      'ショシ'),
            (      '所思',      'ショシ'),
            (      '襲用',    'シュウヨウ'),
            (      '襲爵',   'シュウシャク'),
            ('フラッシングする', 'フラッシングスル'),
            ( 'フラダンスする',  'フラダンススル')],
           names=['headword', 'reading'], length=84644)